# Libraries

In [1]:
import pandas as pd
import glob
import requests
import time

# Data Loading

In [4]:
# load the streaming history data
stream_df = pd.read_csv('datasets/tracks_df_processed.csv')

C:\Users\Kim\AppData\Local\Temp\ipykernel_2228\2996977402.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  stream_df = pd.read_csv('datasets/tracks_df_processed.csv')


In [9]:
print(stream_df.shape)
print(stream_df.columns)

(127961, 12)
Index(['ts', 'platform', 'ms_played', 'track_name', 'artist_name',
       'album_name', 'track_id', 'reason_start', 'reason_end', 'shuffle',
       'skipped', 'offline'],
      dtype='object')


In [6]:
# load the track info data
track_info_df = pd.read_csv('datasets/tracks_info_df.csv')

In [10]:
print(track_info_df.shape)
print(track_info_df.columns)

(5172, 5)
Index(['name', 'artistName', 'release_date', 'popularity', 'duration_ms'], dtype='object')


In [7]:
# load the track features data
track_features_df = pd.read_csv('datasets/tracks_features_df.csv')

In [11]:
print(track_features_df.shape)
print(track_features_df.columns)

(5165, 12)
Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'time_signature'],
      dtype='object')
